In [23]:
import requests
import pandas as pd
import numpy as np
from lxml import etree, html
import jieba

In [11]:
class parser():
    def __init__(self):
        self.tree = None
        self.d = dict()
    
    def fetch(self,n):
        r = requests.get("http://www.judicial.gov.tw/constitutionalcourt/p03_01_printpage.asp?expno={0}".format(n))
        self.tree = html.fromstring(r.content.decode("big5"))
        self.d = dict()
        i = 1
        while True:
            try:
                key =  self.tree.xpath("/html/body/center/table/tr/td/table/tr[2]/td/table/tr[{0}]/td[{1}]".format(i,1))[0].text_content()
                value =  self.tree.xpath("/html/body/center/table/tr/td/table/tr[2]/td/table/tr[{0}]/td[{1}]".format(i,2))[0].text_content()
                self.d[key]=value
                i+=1
            except IndexError:
                break

In [40]:
# punctuations, strange characters
stop = [line.strip().decode('utf-8') for line in open('stop_punc.txt').readlines() ]

for n in range(573,690+1):
    # the period chosen: 573 - 690 號解釋
    p = parser()
    try:
        p.fetch(n)
        pt = p.d[u'解釋文'] + p.d[u'理由書']
        dt = [pt]
    except:
        dt = [""]
    
    sep = []
    for i in range(len(dt)):
        words = [i for i in jieba.cut(dt[i])]
        sep.append("\t".join(words))
    sep_n = pd.Series(sep,name="sep")
    
    sep_no_stop = []
    for i in range(len(dt)):
        words = sep_n[i].split('\t')
        text = " ".join(list(set(words)-set(stop)))
        sep_no_stop.append(text)
    sep_f = pd.Series(sep_no_stop)
    
    sep_f.to_csv("test_d/sep_f_%s.txt" %n, encoding="utf-8", index = False)